# Week 5 - Lab 1: Hybrid Retrieval & Re-ranking

**Duration:** 90-120 minutes  
**Level:** Advanced  
**Prerequisites:** Week 4 RAG fundamentals, Week 5 Lessons 1-2

---

## 🎯 Learning Objectives

In this lab, you will:
- Implement dense (semantic) and lexical (BM25) retrieval
- Build fusion strategies (weighted, RRF) to combine retrievers
- Apply MMR for diversity and deduplication
- Implement LLM-based re-ranking with JSON outputs
- Measure recall@k, precision@k, and latency

---

## 📋 Lab Outline

1. Setup and Data Preparation
2. Exercise 1: Dense Retrieval with Embeddings
3. Exercise 2: Lexical Retrieval with BM25
4. Exercise 3: Hybrid Fusion (Weighted + RRF)
5. Exercise 4: MMR for Diversity
6. Exercise 5: LLM Re-ranking
7. Exercise 6: End-to-End Pipeline Evaluation
8. Bonus Challenge: Parameter Sweep

---

## 1. Setup and Data Preparation

In [ ]:
# Install required packages
!pip install -q openai numpy rank-bm25 python-dotenv

In [ ]:
import os
import json
import time
import numpy as np
from typing import List, Dict, Tuple, Set
from openai import OpenAI
from rank_bm25 import BM25Okapi
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✅ Setup complete!")

### Sample Document Corpus

We'll use a small corpus of documents about RAG systems.

In [ ]:
# Sample corpus
CORPUS = [
    {"id": "doc1", "text": "RAG combines retrieval and generation for accurate, grounded answers. It retrieves relevant documents then generates responses based on that context."},
    {"id": "doc2", "text": "Vector databases like Chroma, Pinecone, and Weaviate enable semantic search through embedding similarity. HNSW indexes provide fast approximate nearest neighbor search."},
    {"id": "doc3", "text": "Hybrid retrieval combines dense vectors with BM25 lexical search. This improves recall on rare terms, IDs, and exact matches that pure semantic search might miss."},
    {"id": "doc4", "text": "Query rewriting with HyDE generates hypothetical answers to improve retrieval. Multi-query expansion creates paraphrases for better coverage."},
    {"id": "doc5", "text": "Re-ranking with cross-encoders or LLMs refines initial retrieval results. This two-stage approach balances speed and precision."},
    {"id": "doc6", "text": "MMR (Maximal Marginal Relevance) promotes diversity in retrieved results. It balances relevance to the query with dissimilarity to already selected documents."},
    {"id": "doc7", "text": "Production RAG systems need monitoring for recall, latency, and cost. SLOs typically target 99.9% availability and p95 latency under 2 seconds."},
    {"id": "doc8", "text": "Chunking strategies affect retrieval quality. Options include fixed-token windows, paragraph-aware splitting, and semantic chunking with overlap."},
    {"id": "doc9", "text": "Embeddings transform text into dense vectors capturing semantic meaning. OpenAI's text-embedding-3-small produces 1536-dimensional vectors efficiently."},
    {"id": "doc10", "text": "Index tuning involves parameters like HNSW's ef_search and M. Higher values improve recall at the cost of increased latency and memory usage."},
]

print(f"Corpus size: {len(CORPUS)} documents")
print(f"Sample doc: {CORPUS[0]['text'][:80]}...")

### Ground Truth for Evaluation

We define relevant documents for test queries.

In [ ]:
# Test queries with ground truth
TEST_QUERIES = [
    {
        "id": "q1",
        "text": "How does hybrid retrieval work?",
        "relevant": {"doc3", "doc2"},
    },
    {
        "id": "q2",
        "text": "What is MMR and why use it?",
        "relevant": {"doc6"},
    },
    {
        "id": "q3",
        "text": "Explain query rewriting techniques",
        "relevant": {"doc4"},
    },
]

print(f"Test queries: {len(TEST_QUERIES)}")

---

## Exercise 1: Dense Retrieval with Embeddings

**Task:** Implement semantic search using OpenAI embeddings and cosine similarity.

**Steps:**
1. Generate embeddings for all corpus documents
2. Implement cosine similarity function
3. Create a dense retriever that returns top-k documents

In [ ]:
def get_embedding(text: str, model: str = "text-embedding-3-small") -> List[float]:
    """Get embedding vector for text."""
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding


def get_embeddings_batch(texts: List[str], model: str = "text-embedding-3-small") -> List[List[float]]:
    """Get embeddings for multiple texts in batch."""
    cleaned = [t.replace("\n", " ") for t in texts]
    response = client.embeddings.create(input=cleaned, model=model)
    return [item.embedding for item in response.data]


# TODO: Generate embeddings for corpus
print("Generating embeddings for corpus...")
corpus_texts = [doc["text"] for doc in CORPUS]
corpus_embeddings = get_embeddings_batch(corpus_texts)
corpus_embeddings = np.array(corpus_embeddings)

print(f"✅ Generated {len(corpus_embeddings)} embeddings, shape: {corpus_embeddings.shape}")

In [ ]:
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate cosine similarity between two vectors."""
    # TODO: Implement cosine similarity
    # Hint: dot product divided by product of norms
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot / (norm1 * norm2)


def dense_retrieve(query: str, k: int = 5) -> List[Tuple[str, float]]:
    """Retrieve top-k documents using dense (semantic) search."""
    # TODO: Implement dense retrieval
    # 1. Get query embedding
    # 2. Calculate similarity with all corpus embeddings
    # 3. Return top-k (doc_id, score) pairs
    
    query_emb = np.array(get_embedding(query))
    
    # Calculate similarities
    similarities = []
    for i, doc_emb in enumerate(corpus_embeddings):
        sim = cosine_similarity(query_emb, doc_emb)
        similarities.append((CORPUS[i]["id"], sim))
    
    # Sort by similarity descending and return top-k
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:k]


# Test dense retrieval
query = "What is hybrid search?"
results = dense_retrieve(query, k=3)

print(f"Query: {query}\n")
print("Dense retrieval results:")
for doc_id, score in results:
    print(f"  {doc_id}: {score:.3f}")

---

## Exercise 2: Lexical Retrieval with BM25

**Task:** Implement keyword-based retrieval using BM25.

**Steps:**
1. Tokenize corpus documents
2. Build BM25 index
3. Create a lexical retriever

In [ ]:
def simple_tokenize(text: str) -> List[str]:
    """Simple tokenization (lowercase + split)."""
    return text.lower().split()


# TODO: Build BM25 index
tokenized_corpus = [simple_tokenize(doc["text"]) for doc in CORPUS]
bm25 = BM25Okapi(tokenized_corpus)

print(f"✅ BM25 index built with {len(tokenized_corpus)} documents")

In [ ]:
def bm25_retrieve(query: str, k: int = 5) -> List[Tuple[str, float]]:
    """Retrieve top-k documents using BM25 lexical search."""
    # TODO: Implement BM25 retrieval
    # 1. Tokenize query
    # 2. Get BM25 scores
    # 3. Return top-k (doc_id, score) pairs
    
    query_tokens = simple_tokenize(query)
    scores = bm25.get_scores(query_tokens)
    
    # Create (doc_id, score) pairs and sort
    results = [(CORPUS[i]["id"], scores[i]) for i in range(len(scores))]
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:k]


# Test BM25 retrieval
query = "What is hybrid search?"
results = bm25_retrieve(query, k=3)

print(f"Query: {query}\n")
print("BM25 retrieval results:")
for doc_id, score in results:
    print(f"  {doc_id}: {score:.3f}")

---

## Exercise 3: Hybrid Fusion (Weighted + RRF)

**Task:** Combine dense and BM25 results using two fusion strategies.

**Steps:**
1. Implement score normalization
2. Implement weighted score fusion
3. Implement Reciprocal Rank Fusion (RRF)

In [ ]:
def normalize_scores(scores: Dict[str, float]) -> Dict[str, float]:
    """Min-max normalize scores to [0, 1]."""
    if not scores:
        return {}
    
    # TODO: Implement min-max normalization
    vals = list(scores.values())
    min_val, max_val = min(vals), max(vals)
    
    if max_val - min_val < 1e-9:
        return {k: 0.0 for k in scores}
    
    return {k: (v - min_val) / (max_val - min_val) for k, v in scores.items()}


def weighted_fusion(
    dense_results: List[Tuple[str, float]],
    bm25_results: List[Tuple[str, float]],
    alpha: float = 0.6,
    k: int = 5
) -> List[Tuple[str, float]]:
    """Fuse results using weighted score combination."""
    # TODO: Implement weighted fusion
    # 1. Convert to dicts
    # 2. Normalize separately
    # 3. Combine: alpha * dense + (1-alpha) * bm25
    # 4. Sort and return top-k
    
    dense_dict = dict(dense_results)
    bm25_dict = dict(bm25_results)
    
    dense_norm = normalize_scores(dense_dict)
    bm25_norm = normalize_scores(bm25_dict)
    
    all_ids = set(dense_norm.keys()) | set(bm25_norm.keys())
    
    fused = {}
    for doc_id in all_ids:
        d_score = dense_norm.get(doc_id, 0.0)
        b_score = bm25_norm.get(doc_id, 0.0)
        fused[doc_id] = alpha * d_score + (1 - alpha) * b_score
    
    results = sorted(fused.items(), key=lambda x: x[1], reverse=True)
    return results[:k]


# Test weighted fusion
query = "What is hybrid search?"
dense_res = dense_retrieve(query, k=10)
bm25_res = bm25_retrieve(query, k=10)
fused_res = weighted_fusion(dense_res, bm25_res, alpha=0.6, k=5)

print(f"Query: {query}\n")
print("Weighted fusion results (alpha=0.6):")
for doc_id, score in fused_res:
    print(f"  {doc_id}: {score:.3f}")

In [ ]:
def rrf_fusion(
    dense_results: List[Tuple[str, float]],
    bm25_results: List[Tuple[str, float]],
    k_param: int = 60,
    top_k: int = 5
) -> List[Tuple[str, float]]:
    """Fuse results using Reciprocal Rank Fusion."""
    # TODO: Implement RRF
    # Formula: score = sum(1 / (k + rank)) for each retriever
    # rank is 1-based position in results list
    
    rrf_scores = {}
    
    # Process dense results
    for rank, (doc_id, _) in enumerate(dense_results, start=1):
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0.0) + 1.0 / (k_param + rank)
    
    # Process BM25 results
    for rank, (doc_id, _) in enumerate(bm25_results, start=1):
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0.0) + 1.0 / (k_param + rank)
    
    results = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    return results[:top_k]


# Test RRF
rrf_res = rrf_fusion(dense_res, bm25_res, k_param=60, top_k=5)

print(f"Query: {query}\n")
print("RRF fusion results (k=60):")
for doc_id, score in rrf_res:
    print(f"  {doc_id}: {score:.3f}")

---

## Exercise 4: MMR for Diversity

**Task:** Implement Maximal Marginal Relevance to promote diversity.

**Steps:**
1. Given initial retrieval results, apply MMR
2. Balance relevance vs diversity with lambda parameter

In [ ]:
def mmr_select(
    query_emb: np.ndarray,
    doc_ids: List[str],
    k: int = 5,
    lambda_param: float = 0.5
) -> List[str]:
    """
    Select k documents using Maximal Marginal Relevance.
    
    Args:
        query_emb: Query embedding vector
        doc_ids: Candidate document IDs
        k: Number of documents to select
        lambda_param: Balance between relevance (1.0) and diversity (0.0)
    """
    # TODO: Implement MMR
    # Algorithm:
    # 1. Start with empty selected set
    # 2. While len(selected) < k:
    #    a. For each candidate, calculate:
    #       MMR = lambda * sim(q, doc) - (1-lambda) * max(sim(doc, selected_doc))
    #    b. Select candidate with highest MMR score
    #    c. Remove from candidates, add to selected
    
    # Get embeddings for candidates
    doc_indices = [i for i, doc in enumerate(CORPUS) if doc["id"] in doc_ids]
    candidate_embs = corpus_embeddings[doc_indices]
    candidate_ids = [CORPUS[i]["id"] for i in doc_indices]
    
    selected = []
    candidates = list(range(len(candidate_ids)))
    
    # Calculate query similarities once
    query_sims = np.dot(candidate_embs, query_emb) / (
        np.linalg.norm(candidate_embs, axis=1) * np.linalg.norm(query_emb)
    )
    
    while candidates and len(selected) < k:
        if not selected:
            # First selection: most relevant
            best_idx = candidates[np.argmax(query_sims[candidates])]
            selected.append(best_idx)
            candidates.remove(best_idx)
        else:
            # Calculate MMR for remaining candidates
            mmr_scores = []
            selected_embs = candidate_embs[selected]
            
            for c in candidates:
                # Relevance to query
                relevance = query_sims[c]
                
                # Max similarity to already selected
                c_emb = candidate_embs[c]
                sims_to_selected = np.dot(selected_embs, c_emb) / (
                    np.linalg.norm(selected_embs, axis=1) * np.linalg.norm(c_emb)
                )
                redundancy = np.max(sims_to_selected)
                
                # MMR score
                mmr = lambda_param * relevance - (1 - lambda_param) * redundancy
                mmr_scores.append(mmr)
            
            # Select best MMR
            best_idx = candidates[np.argmax(mmr_scores)]
            selected.append(best_idx)
            candidates.remove(best_idx)
    
    return [candidate_ids[i] for i in selected]


# Test MMR
query = "What is hybrid search?"
query_emb = np.array(get_embedding(query))
initial_results = [doc_id for doc_id, _ in fused_res[:10]]

mmr_results = mmr_select(query_emb, initial_results, k=5, lambda_param=0.5)

print(f"Query: {query}\n")
print("MMR selected documents (lambda=0.5):")
for i, doc_id in enumerate(mmr_results, 1):
    print(f"  {i}. {doc_id}")

---

## Exercise 5: LLM Re-ranking

**Task:** Use an LLM to re-rank retrieved documents.

**Steps:**
1. Create a re-ranking prompt
2. Parse JSON output with scores
3. Return re-ranked results

In [ ]:
def llm_rerank(
    query: str,
    doc_ids: List[str],
    top_k: int = 5
) -> List[Tuple[str, float]]:
    """
    Re-rank documents using LLM.
    
    Returns:
        List of (doc_id, score) tuples sorted by relevance
    """
    # Get document texts
    candidates = []
    for i, doc in enumerate(CORPUS):
        if doc["id"] in doc_ids:
            candidates.append((i, doc["id"], doc["text"]))
    
    # Build prompt
    system_prompt = (
        "You are a relevance ranking expert. Rank passages by relevance to the query. "
        "Return JSON list with format: [{\"index\": <int>, \"score\": <0-1 float>, \"reason\": \"...\"}] "
        "sorted by score descending."
    )
    
    user_prompt = f"Query: {query}\n\nCandidates:\n"
    for idx, doc_id, text in candidates:
        user_prompt += f"{idx}) {text[:200]}...\n\n"
    
    # Call LLM
    # TODO: Implement LLM call and parse JSON
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
    )
    
    # Parse response
    text = response.choices[0].message.content
    
    # Extract JSON (handle markdown code blocks)
    if "```json" in text:
        text = text.split("```json")[1].split("```")[0]
    elif "```" in text:
        text = text.split("```")[1].split("```")[0]
    
    rankings = json.loads(text.strip())
    
    # Map indices back to doc_ids
    results = []
    for item in rankings[:top_k]:
        idx = item["index"]
        score = item["score"]
        doc_id = candidates[idx][1]
        results.append((doc_id, score))
    
    return results


# Test LLM re-ranking
query = "What is hybrid search?"
candidate_ids = [doc_id for doc_id, _ in fused_res[:8]]

reranked = llm_rerank(query, candidate_ids, top_k=5)

print(f"Query: {query}\n")
print("LLM re-ranked results:")
for doc_id, score in reranked:
    print(f"  {doc_id}: {score:.3f}")

---

## Exercise 6: End-to-End Pipeline Evaluation

**Task:** Build complete pipeline and evaluate with recall@k, precision@k.

**Steps:**
1. Create pipeline function
2. Implement evaluation metrics
3. Run on test queries and report results

In [ ]:
def precision_recall_at_k(
    retrieved: List[str],
    relevant: Set[str],
    k: int
) -> Tuple[float, float]:
    """Calculate precision and recall at k."""
    # TODO: Implement metrics
    topk = retrieved[:k]
    hits = sum(1 for doc_id in topk if doc_id in relevant)
    
    precision = hits / max(1, k)
    recall = hits / max(1, len(relevant))
    
    return precision, recall


def hybrid_rag_pipeline(
    query: str,
    method: str = "rrf",
    k: int = 5,
    use_mmr: bool = False,
    use_rerank: bool = False
) -> List[str]:
    """
    Complete hybrid RAG retrieval pipeline.
    
    Args:
        query: Query text
        method: Fusion method ('weighted' or 'rrf')
        k: Number of results
        use_mmr: Apply MMR for diversity
        use_rerank: Apply LLM re-ranking
    """
    # Step 1: Dense + BM25 retrieval
    dense_res = dense_retrieve(query, k=20)
    bm25_res = bm25_retrieve(query, k=20)
    
    # Step 2: Fusion
    if method == "weighted":
        fused = weighted_fusion(dense_res, bm25_res, alpha=0.6, k=k*2)
    else:
        fused = rrf_fusion(dense_res, bm25_res, k_param=60, top_k=k*2)
    
    doc_ids = [doc_id for doc_id, _ in fused]
    
    # Step 3: Optional MMR
    if use_mmr:
        query_emb = np.array(get_embedding(query))
        doc_ids = mmr_select(query_emb, doc_ids, k=k*2, lambda_param=0.5)
    
    # Step 4: Optional LLM re-ranking
    if use_rerank:
        reranked = llm_rerank(query, doc_ids[:k*2], top_k=k)
        doc_ids = [doc_id for doc_id, _ in reranked]
    
    return doc_ids[:k]


# Evaluate pipeline
def evaluate_pipeline(pipeline_fn, queries, k=5):
    """Evaluate pipeline on test queries."""
    precisions = []
    recalls = []
    
    for query_info in queries:
        query = query_info["text"]
        relevant = query_info["relevant"]
        
        results = pipeline_fn(query, k=k)
        p, r = precision_recall_at_k(results, relevant, k)
        
        precisions.append(p)
        recalls.append(r)
    
    return {
        "precision@k": sum(precisions) / len(precisions),
        "recall@k": sum(recalls) / len(recalls),
    }


# Test different configurations
configs = [
    {"name": "Weighted fusion", "method": "weighted", "use_mmr": False, "use_rerank": False},
    {"name": "RRF fusion", "method": "rrf", "use_mmr": False, "use_rerank": False},
    {"name": "RRF + MMR", "method": "rrf", "use_mmr": True, "use_rerank": False},
]

print("Pipeline Evaluation Results:\n")
for config in configs:
    pipeline = lambda q, k=5: hybrid_rag_pipeline(
        q, method=config["method"], k=k,
        use_mmr=config["use_mmr"], use_rerank=config["use_rerank"]
    )
    
    results = evaluate_pipeline(pipeline, TEST_QUERIES, k=5)
    
    print(f"{config['name']}:")
    print(f"  Precision@5: {results['precision@k']:.3f}")
    print(f"  Recall@5: {results['recall@k']:.3f}")
    print()

---

## Bonus Challenge: Parameter Sweep

**Task:** Sweep fusion weights (alpha) and measure impact on recall.

Try different alpha values (0.3, 0.5, 0.7, 0.9) and plot results.

In [ ]:
# TODO: Implement parameter sweep
# 1. Loop over alpha values
# 2. For each, run evaluation
# 3. Collect and display results

alphas = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
sweep_results = []

for alpha in alphas:
    # Override weighted_fusion to use this alpha
    pipeline = lambda q, k=5: hybrid_rag_pipeline(q, method="weighted", k=k)
    
    # Temporarily modify fusion function
    original_fn = weighted_fusion
    
    def custom_fusion(d, b, alpha=alpha, k=5):
        return original_fn(d, b, alpha=alpha, k=k)
    
    # Run evaluation
    metrics = evaluate_pipeline(pipeline, TEST_QUERIES, k=5)
    sweep_results.append({
        "alpha": alpha,
        "precision": metrics["precision@k"],
        "recall": metrics["recall@k"]
    })

print("Alpha Sweep Results:\n")
print("Alpha | Precision@5 | Recall@5")
print("------|-------------|----------")
for r in sweep_results:
    print(f"{r['alpha']:.1f}   | {r['precision']:.3f}       | {r['recall']:.3f}")

---

## 🎉 Lab Complete!

### What You Learned

- ✅ Dense retrieval with embeddings and cosine similarity
- ✅ Lexical retrieval with BM25
- ✅ Hybrid fusion strategies (weighted and RRF)
- ✅ MMR for diversity and deduplication
- ✅ LLM-based re-ranking with JSON outputs
- ✅ End-to-end pipeline evaluation
- ✅ Parameter tuning and experimentation

### Next Steps

1. Try with your own document corpus
2. Experiment with different fusion weights
3. Add more sophisticated query rewriting
4. Integrate with a real vector database
5. Move on to Lab 2: Index Tuning and Recall Testing

### Resources

- Week 5 Resources: [../resources/README.md](../resources/README.md)
- Hybrid Retrieval Guide: [../resources/hybrid-retrieval-tuning.md](../resources/hybrid-retrieval-tuning.md)
- Evaluation Harness: [../resources/examples/](../resources/examples/)